# [Projet 3: Développez un moteur de recommandations de films](https://openclassrooms.com/projects/developpez-un-moteur-de-recommandations-de-films)

Link to raw data: [here](https://s3-eu-west-1.amazonaws.com/static.oc-static.com/prod/courses/files/Parcours_data_scientist/Projet+-+Moteur+de+recommandation+de+films/imdb-5000-movie-dataset.zip).

In [64]:
import os
HOME = os.path.expanduser('~/')
HOST = os.uname()[1]
if HOST == 'Arthurs-MacBook-Pro.local':
    os.chdir(HOME+'/Documents/GitHub/OCDataSciencePath/Project3/')    # @home
elif HOST == 'Sirius.local':
    os.chdir(HOME+'Perso/GitHub/OCDataSciencePath/Project3/')         # @L2
else:
    raise ValueError('unknown host: {}'.format(HOST))
    
import numpy as np
import pandas as pd

In [2]:
if HOST == 'Arthurs-MacBook-Pro.local':
    pathToData = HOME+'xxx'    # @home
elif HOST == 'Sirius.local':
    pathToData = HOME+'Downloads/movie_metadata.csv'         # @L2
else:
    raise ValueError('unknown host: {}'.format(HOST))
    
df = pd.read_csv(pathToData)

In [53]:
n,p = df.shape
print(n,p)
df.sample(5)

5043 28


,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
2798,Color,Shawn Levy,69.0,88.0,189.0,799.0,Donald Faison,934.0,47811275.0,Adventure|Comedy|Family,...,99.0,English,USA,PG,15000000.0,2002.0,927.0,5.4,1.85,896
4982,Color,Daniel Schechter,15.0,87.0,8.0,272.0,Kevin Corrigan,969.0,NaN,Comedy,...,4.0,English,USA,Not Rated,NaN,2012.0,778.0,6.3,NaN,209
4395,Color,Eytan Fox,51.0,90.0,21.0,27.0,Lior Ashkenazi,53.0,155972.0,Comedy|Drama|Romance,...,23.0,Hebrew,Israel,Unrated,1500000.0,2006.0,30.0,7.3,NaN,0
3985,Color,Jim Abrahams,134.0,88.0,104.0,318.0,Lloyd Bridges,628.0,83400000.0,Comedy,...,427.0,English,USA,PG,3500000.0,1980.0,575.0,7.8,1.85,16000
3680,Color,Spike Lee,103.0,120.0,0.0,388.0,John Savage,782.0,27545445.0,Drama,...,418.0,English,USA,R,6500000.0,1989.0,652.0,7.9,1.85,0


In [54]:
for col in df.columns.sort_values():
    miss = 100 * (1 - df[col].count()/n)
    print('{:>30}: {}%'.format(col, miss))

        actor_1_facebook_likes: 0.13880626611144642%
                  actor_1_name: 0.13880626611144642%
        actor_2_facebook_likes: 0.25778306563553066%
                  actor_2_name: 0.25778306563553066%
        actor_3_facebook_likes: 0.4560777315090192%
                  actor_3_name: 0.4560777315090192%
                  aspect_ratio: 6.52389450723776%
                        budget: 9.756097560975608%
     cast_total_facebook_likes: 0.0%
                         color: 0.376759865159626%
                content_rating: 6.008328375966688%
                       country: 0.09914733293674427%
       director_facebook_likes: 2.0622645250842786%
                 director_name: 2.0622645250842786%
                      duration: 0.2974419988102328%
          facenumber_in_poster: 0.25778306563553066%
                        genres: 0.0%
                         gross: 17.52924846321634%
                    imdb_score: 0.0%
                      language: 0.2379535990481907%
     

We start with simple variables:

In [77]:
keepVar = ['movie_title',
           'actor_1_name','actor_2_name','actor_3_name']
wdf = df[keepVar].copy()
wdf.sample(5)

,movie_title,actor_1_name,actor_2_name,actor_3_name
2560,Vicky Cristina Barcelona,Scarlett Johansson,Kevin Dunn,Christopher Evan Welch
3903,The Great Escape,James Coburn,Donald Pleasence,Gordon Jackson
2303,Perception,LeVar Burton,Eric McCormack,Scott Wolf
2061,The Adventurer: The Curse of the Midas Box,Ioan Gruffudd,Keeley Hawes,Oliver Stark
276,Stuart Little,Chazz Palminteri,Nathan Lane,Jeffrey Jones


Needed post-processing:
- lowerise all text to avoid missing doublons
- doublons/approximative doublons in `movie_title`,`actor_N_name`
- try one-hot encode the three `actor_N_name` (ignore order)


In [78]:
# lowerise

toLowerVar = ('movie_title','actor_1_name','actor_2_name','actor_3_name')
for col in toLowerVar:
    wdf[col] = wdf[col].str.lower()
wdf.sample(5)

,movie_title,actor_1_name,actor_2_name,actor_3_name
4229,friday the 13th: a new beginning,tiffany helm,melanie kinnaman,dominick brascia
4235,chicken tikka masala,saeed jaffrey,zohra segal,shobu kapoor
1254,paul,bobby lee,nelson ascencio,jeremy owen
4659,a separation,shahab hosseini,leila hatami,peyman moaadi
1918,beauty shop,alfre woodard,keshia knight pulliam,lisaraye mccoy


In [79]:
# remove pure duplicates

print(wdf.duplicated().sum())
# wdf.loc[wdf.duplicated()].sort_values(by='movie_title')
wdf.drop_duplicates(inplace=True)

123


In [80]:
# look specifically at the rows where only film title is duplicated

print(wdf['movie_title'].duplicated(keep=False).sum())
wdf.loc[wdf['movie_title'].duplicated(keep=False)].sort_values(by='movie_title')

6


,movie_title,actor_1_name,actor_2_name,actor_3_name
1852,brothers,natalie portman,jake gyllenhaal,bailee madison
2882,brothers,natalie portman,jake gyllenhaal,ethan suplee
3791,out of the blue,raymond burr,don gordon,jim byrnes
3840,out of the blue,william kircher,matthew sunderland,paul glover
1002,the host,j.d. evermore,chandler canterbury,rachel roberts
2988,the host,doona bae,kang-ho song,ah-sung ko


In [81]:
# no obvious error: don't want to check which one is correct... just drop them

wdf.drop_duplicates(subset='movie_title',inplace=True)
wdf.sample(5)

,movie_title,actor_1_name,actor_2_name,actor_3_name
3936,the loss of sexual innocence,kelly macdonald,saffron burrows,julian sands
4892,the stewardesses,christina hart,nancy ison,william condos
4251,it follows,maika monroe,ruby harris,jake weary
1319,disturbia,sarah roemer,aaron yoo,jose pablo cantillo
4891,urbania,dan futterman,josh hamilton,matt keeslar


In [84]:
# one hot encode the names

toOneHotVar = ('actor_1_name','actor_2_name','actor_3_name')
allActor = []
for col in toOneHotVar:
    allActor += list(wdf[col])
allActor = np.unique(allActor)

In [85]:
for a in allActor: # speed this up!
    value = np.full((wdf.shape[0],),False)
    for col in toOneHotVar:
        value = value & (wdf[col] == a)    
    wdf[a] = 1*value

In [89]:
wdf.drop(list(toOneHotVar),axis=1,inplace=True)
wdf.sample(5)

,movie_title,50 cent,a. michael baldwin,a.j. buckley,a.j. delucia,a.j. langer,aaliyah,aaron ashmore,aaron hill,aaron hughes,...,zoë bell,zoë kravitz,zoë poledouris,zubaida sahar,zuhair haddad,álex angulo,ángela molina,émilie dequenne,ólafur darri ólafsson,óscar jaenada
4760,scott walker: 30 century man,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3512,cyrus,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4535,taxi driver,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1378,twilight,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
545,munich,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Dump the table

In [ ]:
path,nameext= os.path.split(pathToData)
name,ext = os.path.splitext(nameext)

wdf.to_csv(os.path.join(path,name+'_CLEAN'+ext))